In [18]:
import csv

# TODO: Franklin -- clean up everything below to use the transform_helpers library
from tools import transform_helpers

In [19]:
### We use this to map cities to counties ####

city_county_dict = dict()
with open('../data/mapping_files/us_cities_to_county.csv') as city_mapping_file:
    city_mapping_reader = csv.DictReader(city_mapping_file)
    for c in city_mapping_reader:
        # need to add in state, otherwise get missed mappings
        city_county_dict[(c['city'] + ' ' + c['state_id']).lower()] = c['county_name']

FileNotFoundError: [Errno 2] No such file or directory: '../data/mapping_files/us_cities_to_county.csv'

In [16]:
import pandas as pd

sundown_data = {}
with open('../data/extracts/sundown_towns.csv') as sundown_file:
    city_reader = csv.DictReader(sundown_file)
    for r in city_reader:
        city_state = r['city'] + ' ' + r['state']
        if city_state.lower() in city_county_dict.keys():
            data = r
            data['county'] = city_county_dict[city_state.lower()] + ' County' +', ' + r['state']
            sundown_data[r['id']] = data
        elif 'County' in r['city']:
            # print('missed mapping: {}'.format(r['city']))
            data['county'] = r['city']
            sundown_data[r['id']] = data

In [27]:
def get_county_from_city(city, state, city_county_map):
    city_to_county_output = {}
    lookup_str = ' '.join([city, state]).lower()
    if lookup_str in city_county_map.keys():
        return city_county_map[lookup_str]
    else:
        print('missed mapping: ', city)

In [28]:
# add county to city_shootings csv
city_shootings = pd.read_csv('../data/extracts/police-killings-city.csv')
city_shootings['county'] = city_shootings.apply(lambda r: get_county_from_city(r.City, r.State, city_county_dict) + ' County, {}'.format(r.State), axis=1)

In [30]:
city_shootings.to_csv('../data/outputs/police-killings-county.csv')